In [ ]:
#start code for pagerank visualization using panda to draw bar_chart
new_panda = name_basics[['nconst','primaryName']].copy() #get copy of nconst and Name from big panda
#print new_panda
#new_panda.set_index('nconst',inplace=True)
#print new_panda

selected_panda = new_panda[new_panda['nconst'].isin(pr.keys())] #select rows with nconst in pagerank panda
print selected_panda #panda with nconst matching Name, 255 selected by pagerank. Done


pr_panda['nconst'] = pr_panda.index #make nconst from index into a column in panda
#pr_panda.columns = ['nconst','value']

#print pr_panda

#start to draw the graph
For_bar_panda1 = pd.merge(pr_panda,selected_panda,
                 on='nconst')
#names_select.drop(columns =['knownForTitles'], inplace = True) 
#print For_bar_panda1
#For_bar_panda1.drop('nconst',axis =1)
For_bar_panda1.drop(columns =['nconst'], inplace = True)#now panda for bar chart is done

#print For_bar_panda1
For_bar_panda1.columns=['val', 'primaryName'] #change the column name, prepare for bar chart
For_bar_panda1
For_bar_panda1.plot.bar(x='primaryName', y='val', rot=90, fontsize = 1)
#plt.savefig("nx.png", format="PNG")

print Bar_chart #see the graph, too many stars

#2nd graph next, filter the data
For_bar_panda2 = For_bar_panda1.copy()
#For_bar_panda2.sort_values(by=['val'],ascending = False) #sort the pandas by val, high to low (Note: problem!!)
#print For_bar_panda2
#For_bar_panda3 = For_bar_panda2.head(20)
#print For_bar_panda3 #get the top 20 rank Names
#Bar_chart2 = For_bar_panda3.plot.bar(x='primaryName', y='val', rot=90, fontsize = 10)
star = For_bar_panda2['val']>= 0.020000 #filter to get val higher than 0.02
For_bar_panda3 = For_bar_panda2[star] #contain 19 Names
Bar_chart2 = For_bar_panda3.plot.bar(x='primaryName', y='val', rot=90, fontsize = 10) #Done, bar for top 19 stars
#end code for pagerank visulizatoin using bar_chart


#start code for network
#This part is to create a dict matching nconst to Names, Names from the selected 225 people
Name_dic_panda = selected_panda.copy() #selected_panda in lower part of codes, used in Bar_chart plotting first
#print Name_dic_panda
Name_dic = Name_dic_panda.set_index('nconst').to_dict() #convert panda to dict
Name_dic = Name_dic['primaryName']#now the Name_dic is done, key be nconst, val be Name

#modify nx_merge, change all nconst into REAL Names
nx_merge_Name = nx_merge.replace({"nconst_x": Name_dic}).replace({"nconst_y": Name_dic}).copy()
nx_merge_Name1 = nx_merge_Name.copy()
nx_merge_Name1['nconst_x'] = nx_merge_Name1['nconst_x'].replace(' ', '', regex=True)
nx_merge_Name1['nconst_y'] = nx_merge_Name1['nconst_y'].replace(' ', '', regex=True)

nx_merge_Name1 #success!, now use nx_merge_Name to draw network

#Jiarui
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

g1 = nx.from_pandas_edgelist(nx_merge_Name1, source = 'nconst_x', target = 'nconst_y')

#next, import new module called bokeh
from bokeh.io import output_file, show
from bokeh.models import CustomJSTransform, LabelSet
from bokeh.models.graphs import from_networkx
from bokeh.plotting import figure
from bokeh.transform import transform    
import networkx as nx
import matplotlib.pyplot as plt

g = nx.from_pandas_edgelist(nx_merge, source = 'nconst_x', target = 'nconst_y') #did not use 

#Jiarui test, use g1 instead, nodes have Name as label

p = figure(x_range=(-3,3), y_range=(-3,3))
p.grid.grid_line_color = None

r = from_networkx(g1, nx.spring_layout, scale=3, center=(0,0)) # here use g1 instead of g
r.node_renderer.glyph.size=10
r.edge_renderer.glyph.line_alpha=0.4

p.renderers.append(r)
source = r.node_renderer.data_source
source.data['names'] = [str(x) for x in source.data['index']]

# create a transform that can extract the actual x,y positions
code = """
    var result = new Float64Array(xs.length)
    for (var i = 0; i < xs.length; i++) {
        result[i] = provider.graph_layout[xs[i]][%s]
    }
    return result
"""
xcoord = CustomJSTransform(v_func=code % "0", args=dict(provider=r.layout_provider))
ycoord = CustomJSTransform(v_func=code % "1", args=dict(provider=r.layout_provider))

# Use the transforms to supply coords to a LabelSet 
labels = LabelSet(x=transform('index', xcoord),
                  y=transform('index', ycoord),
                  text='names', text_font_size="5px",
                  x_offset=5, y_offset=5,
                  source=source, render_mode='canvas')

p.add_layout(labels)

show(p)
#end code for Network visualization